In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/home/martyn/anaconda3/envs/DataScience/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
tf.__version__

'2.2.0'

In [3]:
myMinMaxScaler = MinMaxScaler()

In [4]:
import_df = pd.read_csv('20171021_20171124_1min.csv', index_col = 0, date_parser = pd.to_datetime)
import_df

,open,high,low,close,average
timestamp,,,,,
2017-10-21 00:00:00+00:00,6018.7,6022.1,6011.0,6020.0,6016.795301
2017-10-21 00:01:00+00:00,6019.9,6020.0,6014.2,6014.3,6018.435888
2017-10-21 00:02:00+00:00,6014.3,6016.6,6012.1,6012.1,6014.576291
2017-10-21 00:03:00+00:00,6012.1,6014.2,6012.1,6013.0,6012.818012
2017-10-21 00:04:00+00:00,6012.9,6018.0,6012.8,6015.8,6017.010478
...,...,...,...,...,...
2017-11-24 23:55:00+00:00,8221.0,8221.0,8220.5,8221.0,8220.783234
2017-11-24 23:56:00+00:00,8221.0,8222.5,8218.0,8218.0,8220.265747
2017-11-24 23:57:00+00:00,8218.0,8218.0,8211.0,8211.5,8213.875742


In [5]:
len_df = import_df.shape[0]
test_size = int(len_df/3)
train_size = len_df - test_size

In [6]:
train_data = import_df[:train_size]
test_data = import_df[train_size:]

In [7]:
train_scaled = myMinMaxScaler.fit_transform(train_data)
train_scaled

array([[0.25456755, 0.24981517, 0.2544178 , 0.25405068, 0.2516586 ],
       [0.25503803, 0.24899802, 0.25568003, 0.25182524, 0.25229966],
       [0.25284247, 0.24767501, 0.25485169, 0.25096631, 0.25079151],
       ...,
       [0.35387752, 0.34705631, 0.35618492, 0.35146215, 0.35060114],
       [0.35289736, 0.34608351, 0.35283212, 0.34892437, 0.34876917],
       [0.35034894, 0.34627807, 0.35381824, 0.35048608, 0.34945387]])

In [8]:
window = 10
y_train = []
X_train = []
for i in range(window, train_size):
    X_train.append(train_scaled[i-window:i])
    y_train.append(train_scaled[i, 3]) # y as close price to start

In [9]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [10]:
X_train.shape

(33590, 10, 5)

In [11]:
y_train.shape

(33590,)

In [12]:
y_train.astype('float32')

array([0.2430016 , 0.24159606, 0.24307969, ..., 0.35146216, 0.34892437,
       0.35048607], dtype=float32)

In [13]:
regressor = Sequential()
regressor.add(LSTM(5, activation='sigmoid', return_sequences=True, input_shape = (X_train.shape[1],X_train.shape[2])))
regressor.add(Dropout(0.05))
# regressor.add(BatchNormalization(epsilon=1e-06))
# regressor.add(LSTM(60, activation = 'relu', return_sequences=True))
# regressor.add(Dropout(0.3))
# regressor.add(LSTM(80, activation = 'relu', return_sequences=True))
# regressor.add(Dropout(0.4))
# regressor.add(LSTM(120, activation = 'relu'))
regressor.add(LSTM(5, activation='sigmoid' ))
regressor.add(Dropout(0.1))
regressor.add(Dense(units = 1))

In [14]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 5)             220       
_________________________________________________________________
dropout (Dropout)            (None, 10, 5)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5)                 220       
_________________________________________________________________
dropout_1 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 446
Trainable params: 446
Non-trainable params: 0
_________________________________________________________________


In [15]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')

In [16]:

regressor.fit(X_train, y_train, epochs = 10, batch_size = 10)

Epoch 1/10
1275/3359 [==========>...................] - ETA: 7s - loss: nan

KeyboardInterrupt: 